In [1]:
import pandas as pd
import matplotlib.pyplot as plt
from functools import reduce
import plotly.express as px
import numpy as np
import plotly.graph_objects as go
import json

Below we standardise file formatting

In [2]:
code_to_desc = {}

for year in range(2015, 2026):
    if year > 2022:
        df = pd.read_excel(f"./Data/Income/Gross_Pay_{year}.xlsx",sheet_name=1).iloc[3:]
    else:
        df = pd.read_excel(f"./Data/Income/Gross_Pay_{year}.xls",sheet_name=1).iloc[3:]
    df.columns = df.iloc[0]
    df.columns = df.columns.astype(str)
    code_to_desc.update(df.set_index('Code')['Description'].to_dict())
    df = df.rename_axis("Council Code", axis=1)
    df = df.dropna(subset=['Code'])
    print (df.columns)
    df = df.drop(columns=["Description", "(thousand)", "change", "Mean", "25", "75"], axis=1)
    df = df.rename(columns={'Median': "50"})
    df.set_index("Code", inplace=True)
    df = df.iloc[:, :-3]
    df.index.name = None
    df = df[["10", "20", "30", "40", "50", "60", "70", "80", "90"]]
    df.to_csv(f"./Temp/Gross_Pay_{year}.csv", index=True)   

with open("./CleanedData/Council_to_Code.json", "w") as f:
    json.dump(code_to_desc, f, indent=4)

Index(['Description', 'Code', '(thousand)', 'Median', 'change', 'Mean',
       'change', '10', '20', '25', '30', '40', '60', '70', '75', '80', '90',
       'nan', 'nan', 'nan'],
      dtype='object', name='Council Code')
Index(['Description', 'Code', '(thousand)', 'Median', 'change', 'Mean',
       'change', '10', '20', '25', '30', '40', '60', '70', '75', '80', '90',
       'nan', 'nan', 'nan'],
      dtype='object', name='Council Code')
Index(['Description', 'Code', '(thousand)', 'Median', 'change', 'Mean',
       'change', '10', '20', '25', '30', '40', '60', '70', '75', '80', '90',
       'nan', 'nan', 'nan'],
      dtype='object', name='Council Code')
Index(['Description', 'Code', '(thousand)', 'Median', 'change', 'Mean',
       'change', '10', '20', '25', '30', '40', '60', '70', '75', '80', '90',
       'nan', 'nan', 'nan'],
      dtype='object', name='Council Code')
Index(['Description', 'Code', '(thousand)', 'Median', 'change', 'Mean',
       'change', '10', '20', '25', '30', '40

Next we load each dataframe into a list 

In [3]:
dfs = [pd.read_csv(f"./Temp/Gross_Pay_{year}.csv") for year in range(2016, 2026)]

for i, df in enumerate(dfs):
    df = dfs[i]
    df.set_index("Unnamed: 0", inplace=True)
    df = df.tail(-1)
    df.index.name = None
    df = df.rename_axis("Council Code", axis=1)
    dfs[i] = df

Next we take the common council codes for each dataframe so each dataframe has the same dimensions

In [4]:

common_idx = reduce(lambda a, b: a.intersection(b.index), dfs, dfs[0].index)
dfs = [df.loc[common_idx] for df in dfs]
dfs[-1]

Council Code,10,20,30,40,50,60,70,80,90
K02000001,11425,18560,24532,28591,32890,38000,44500,52809,69381
K03000001,11456,18613,24580,28646,32972,38061,44629,52929,69750
K04000001,11424,18589,24563,28627,32991,38058,44677,53162,70250
E92000001,11439,18653,24669,28769,33142,38292,44962,53630,71090
E12000001,10727,17213,22769,26092,29266,33006,38097,45254,56226
...,...,...,...,...,...,...,...,...,...
S12000029,10510,16577,22873,27610,31984,36257,42531,49383,x
S12000030,13366,20888,25654,30216,34244,37413,44045,50139,x
S12000039,11013,16971,22896,26576,30033,34887,41103,46589,x
S12000040,12198,19715,24994,28624,32535,37660,43582,50103,x


In [5]:
years = list(range(2016, 2026))
combined_df = pd.concat(dfs, axis=1)
combined_df.columns = pd.MultiIndex.from_product([years, dfs[0].columns], names=['Year', 'Analysis'])
combined_df.to_csv(f"./CleanedData/IncomeData.csv", index=True)  


In [6]:
combined_df = combined_df.rename(index=code_to_desc)
combined_df.index = combined_df.index.str.strip()
combined_df.sort_index(inplace=True)
combined_df

Year                      2016                                            \
Analysis                    10     20     30     40     50     60     70   
Aberdeen City                x  13839  18469  22476  26507  31263  37210   
Aberdeenshire                x   9405  12749  16731  20456  24688  29589   
Adur                         x      x      x      x      x      x      x   
Amber Valley              9416  13490  16230  19898  23197  27735  31332   
Angus                     7409  11734  15517  17604  20467  23839  26642   
...                        ...    ...    ...    ...    ...    ...    ...   
Wychavon                     x      x      x  16921  20864  23442  27868   
Wyre                         x   8542  11434  15281  18155  19449  23791   
Wyre Forest                  x      x      x      x      x      x      x   
York UA                   7019  11958  15420  18240  21670  25045  29880   
Yorkshire and The Humber  7091  11018  14855  17944  21215  24808  29283   

Year                                    2017  ...   2024   2025                \
Analysis                     80     90    10  ...     90     10     20     30   
Aberdeen City             45487      x  7424  ...      x  12521  21151  26166   
Aberdeenshire             35388      x  6480  ...      x   9812  15892  22900   
Adur                          x      x     x  ...      x      x  12564  18100   
Amber Valley              39645      x  7759  ...      x  12570  18433  23271   
Angus                     29806      x     x  ...      x      x  14207  19290   
...                         ...    ...   ...  ...    ...    ...    ...    ...   
Wychavon                      x      x  7663  ...      x  11166  19993  24996   
Wyre                          x      x     x  ...      x      x  15251  20382   
Wyre Forest                   x      x     x  ...      x      x  15381  20861   
York UA                   36288      x     x  ...      x      x  18228  24917   
Yorkshire and The Humber  34845  43400  7502  ...  57023  11266  17795  23540   

Year                                                                
Analysis                     40     50     60     70     80     90  
Aberdeen City             30249  35483  42605  48795  57190      x  
Aberdeenshire             27478  31551  36743  43604  51817      x  
Adur                      22903  28238  35178      x      x      x  
Amber Valley              26852  31100  35307  38948  46309      x  
Angus                     24551  27520  31569  36992  46567      x  
...                         ...    ...    ...    ...    ...    ...  
Wychavon                  27194  32022  36325  41273  47320      x  
Wyre                      24491  27657  29924  34803      x      x  
Wyre Forest               25219  27346  29584  35842      x      x  
York UA                   28003  32151  37407  44894  55662      x  
Yorkshire and The Humber  26801  30486  34802  39787  47351  59954  

[378 rows x 90 columns]

In [ ]:
import pandas as pd
import numpy as np

df = combined_df.copy()
df = df.replace("x", np.nan)  
df = df.apply(pd.to_numeric)

years = sorted(list(set(df.columns.get_level_values(0))))
percentiles = ['10','20','30','40','50','60','70','80','90']

rows_drop_backwards = []

for idx, row in df.iterrows():

    for p in ['10','90']:
        last_value = None
        nan_run = 0

        for y in years:
            col = (y, p)

            if col not in df.columns:
                continue

            val = row[col]

            if pd.isna(val):
                nan_run += 1

                if nan_run >= 2:
                    rows_drop_backwards.append(idx)
                    break

                if last_value is not None:
                    df.at[idx, col] = last_value

            else:
                last_value = val
                nan_run = 0

        if idx in rows_drop_backwards:
            break

df = df.drop(rows_drop_backwards)
rows_drop_middle = []

for idx, row in df.iterrows():
    for p in ['50','60','70','80','90']:

        nan_run = 0

        for y in years:
            col = (y, p)

            if col not in df.columns:
                continue

            if pd.isna(row[col]):
                nan_run += 1
            else:
                nan_run = 0

            if nan_run >= 2:
                rows_drop_middle.append(idx)
                break

        if idx in rows_drop_middle:
            break

df = df.drop(rows_drop_middle)
def nearest_left(series, pos):
    for i in range(pos - 1, -1, -1):
        if pd.notna(series.iloc[i]):
            return series.iloc[i]
    return None

def nearest_right(series, pos):
    for i in range(pos + 1, len(series)):
        if pd.notna(series.iloc[i]):
            return series.iloc[i]
    return None


rows_drop_sideways = []

for idx, row in df.iterrows():

    row_vals = row.values
    s = row.copy()
    row_pos = df.index.get_loc(idx)

    for col_i in range(len(row_vals)):

        if pd.isna(row_vals[col_i]):

            left = nearest_left(s, col_i)
            right = nearest_right(s, col_i)
            if left is None and right is None:
                rows_drop_sideways.append(idx)
                break
            if left is not None and right is not None:
                val = (left + right) / 2
            elif left is not None:
                val = left
            else:
                val = right

            df.iat[row_pos, col_i] = val

df = df.drop(rows_drop_sideways)

cleaned_df = df.copy()
cleaned_df



/var/folders/dw/62r7df3d26zfcqkyytc46c880000gn/T/ipykernel_90009/301998454.py:14: FutureWarning:

errors='ignore' is deprecated and will raise in a future version. Use to_numeric without passing `errors` and catch exceptions explicitly instead



Year                              2016                                      \
Analysis                            10       20       30       40       50   
Birmingham                      8055.0  12848.0  16749.0  20191.0  23788.0   
City of Edinburgh               9279.0  15113.0  19413.0  22571.0  25971.0   
Cornwall UA                     5850.0   9624.0  12667.0  15536.0  17868.0   
Derbyshire                      7200.0  10943.0  14359.0  17601.0  20927.0   
Devon                           6603.0  10588.0  14582.0  17756.0  20726.0   
East                            6515.0  10792.0  14927.0  18622.0  22330.0   
East Midlands                   7298.0  11449.0  14991.0  18014.0  21122.0   
England                         7465.0  11977.0  16000.0  19640.0  23334.0   
England and Wales               7468.0  11958.0  15944.0  19530.0  23178.0   
Essex                           6182.0  10303.0  14340.0  17967.0  21387.0   
Gloucestershire                 7570.0  11254.0  15504.0  19168.0  22634.0   
Great Britain                   7500.0  11986.0  15961.0  19524.0  23162.0   
Greater Manchester Met County   7337.0  11891.0  15586.0  18882.0  22134.0   
Hampshire                       8000.0  12672.0  17107.0  20838.0  24556.0   
Inner London                   12208.0  20000.0  25494.0  30432.0  35467.0   
Kent                            6789.0  10726.0  14646.0  18198.0  21705.0   
Lancashire                      7329.0  11388.0  15049.0  18054.0  21008.0   
Leeds                           8085.0  13467.0  17347.0  20645.0  24183.0   
Leicestershire                  6191.0  10509.0  14475.0  18050.0  20994.0   
Liverpool                       7856.0  12751.0  16228.0  19851.0  22990.0   
London                          9385.0  16134.0  21785.0  26543.0  31357.0   
Merseyside Met County           7020.0  11019.0  15090.0  18012.0  21521.0   
Norfolk                         5986.0   9617.0  13364.0  16383.0  19929.0   
North East                      7451.0  11647.0  15188.0  18225.0  21177.0   
North West                      7200.0  11434.0  15288.0  18418.0  21664.0   
Northern Ireland                7324.0  11079.0  14535.0  17794.0  20966.0   
Outer London                    7689.0  12173.0  16806.0  21058.0  25250.0   
Oxfordshire                     8955.0  14941.0  19503.0  23573.0  27490.0   
Scotland                        7846.0  12254.0  16117.0  19500.0  22913.0   
South East                      7620.0  12397.0  16710.0  20557.0  24514.0   
South West                      6442.0  10611.0  14755.0  18040.0  21295.0   
South Yorkshire Met County      7197.0  10904.0  14500.0  17413.0  20590.0   
Staffordshire                   7005.0  11077.0  14371.0  17587.0  20858.0   
Suffolk                         6464.0  10310.0  14224.0  17765.0  21117.0   
Tyne and Wear Met County        8029.0  12279.0  15648.0  18717.0  21590.0   
United Kingdom                  7498.0  11952.0  15914.0  19466.0  23084.0   
Wales                           7508.0  11548.0  15056.0  18116.0  20818.0   
Warwickshire                    7925.0  12726.0  16266.0  20134.0  23169.0   
West Midlands                   7205.0  11329.0  15022.0  18301.0  21732.0   
West Midlands Met County        7620.0  11986.0  15608.0  18989.0  22501.0   
West Yorkshire Met County       7631.0  11863.0  15576.0  18776.0  22151.0   
Yorkshire and The Humber        7091.0  11018.0  14855.0  17944.0  21215.0   

Year                                                                  2017  \
Analysis                            60       70       80       90       10   
Birmingham                     27714.0  32959.0  38544.0  47934.0   7955.0   
City of Edinburgh              30300.0  35922.0  42850.0  57613.0   9525.0   
Cornwall UA                    21027.0  24795.0  29305.0  17671.5   6038.0   
Derbyshire                     24306.0  28553.0  34126.0  43291.0   6897.0   
Devon                          23747.0  27786.0  32728.0  40729.0   6878.0   
East                 

In [8]:
combined_percentiles = combined_df.loc[:, [col for col in combined_df.columns if col[1] not in ['Mean']]]
combined_percentiles = combined_percentiles.apply(pd.to_numeric, errors='coerce')



In [9]:
combined_median = combined_df.loc[:, [col for col in combined_df.columns if col[1]  in ['50']]]
combined_median = combined_median.apply(pd.to_numeric, errors='coerce')

combined_median

Year,2016,2017,2018,2019,2020,2021,2022,2023,2024,2025
Analysis,50,50,50,50,50,50,50,50,50,50
Aberdeen City,26507.0,25349.0,26087.0,27435.0,27970.0,26844.0,29429.0,33319.0,35160.0,35483.0
Aberdeenshire,20456.0,20726.0,22702.0,23064.0,23814.0,23473.0,25061.0,26724.0,29736.0,31551.0
Adur,NaN,22762.0,22914.0,21447.0,22877.0,22977.0,23563.0,28029.0,27753.0,28238.0
Amber Valley,23197.0,24060.0,22683.0,24950.0,24450.0,23301.0,23924.0,26342.0,29414.0,31100.0
Angus,20467.0,20022.0,21112.0,NaN,21785.0,22779.0,25822.0,26604.0,28748.0,27520.0
...,...,...,...,...,...,...,...,...,...,...
Wychavon,20864.0,21401.0,22810.0,22525.0,22543.0,24343.0,NaN,29891.0,31721.0,32022.0
Wyre,18155.0,NaN,17902.0,19227.0,20064.0,20985.0,23043.0,24953.0,26691.0,27657.0
Wyre Forest,NaN,18181.0,18158.0,20783.0,20293.0,22665.0,23501.0,NaN,25605.0,27346.0


In [ ]:

median_df = combined_median.xs('50', axis=1, level=1).copy()
median_df = median_df.apply(pd.to_numeric, errors='coerce')

rows_with_adjacent_nans = []
for idx, row in median_df.iterrows():
    row_is_nan = row.isna().values
    has_adjacent = False

    for i in range(len(row_is_nan) - 1):
        if row_is_nan[i] and row_is_nan[i+1]:
            has_adjacent = True
            break

    if has_adjacent:
        rows_with_adjacent_nans.append(idx)

clean_df = median_df.drop(rows_with_adjacent_nans).copy()

def find_left_value(series, pos):
    """Find nearest non-NaN to the left."""
    for i in range(pos - 1, -1, -1):
        if pd.notna(series.iloc[i]):
            return series.iloc[i]
    return None

def find_right_value(series, pos):
    """Find nearest non-NaN to the right."""
    for i in range(pos + 1, len(series)):
        if pd.notna(series.iloc[i]):
            return series.iloc[i]
    return None

rows_to_drop = []

for idx in clean_df.index:
    row = clean_df.loc[idx]

    for col_pos in range(len(row)):
        if pd.isna(row.iloc[col_pos]):

            left_val = find_left_value(row, col_pos)
            right_val = find_right_value(row, col_pos)
            if left_val is None and right_val is None:
                rows_to_drop.append(idx)
                break
            if left_val is not None and right_val is not None:
                new_value = (left_val + right_val) / 2.0
            elif left_val is not None:
                new_value = left_val
            else:
                new_value = right_val
            clean_df.at[idx, row.index[col_pos]] = new_value
clean_df = clean_df.drop(rows_to_drop)
for col in clean_df.columns:
    combined_median[(col, '50')] = combined_median.index.map(
        lambda x: clean_df.at[x, col] if x in clean_df.index else np.nan
    )

combined_median = combined_median.loc[clean_df.index].copy()

In [11]:
combined_median

Year,2016,2017,2018,2019,2020,2021,2022,2023,2024,2025
Analysis,50,50,50,50,50,50,50,50,50,50
Aberdeen City,26507.0,25349.0,26087.0,27435.0,27970.0,26844.0,29429.0,33319.0,35160.0,35483.0
Aberdeenshire,20456.0,20726.0,22702.0,23064.0,23814.0,23473.0,25061.0,26724.0,29736.0,31551.0
Adur,22762.0,22762.0,22914.0,21447.0,22877.0,22977.0,23563.0,28029.0,27753.0,28238.0
Amber Valley,23197.0,24060.0,22683.0,24950.0,24450.0,23301.0,23924.0,26342.0,29414.0,31100.0
Angus,20467.0,20022.0,21112.0,21448.5,21785.0,22779.0,25822.0,26604.0,28748.0,27520.0
...,...,...,...,...,...,...,...,...,...,...
Wychavon,20864.0,21401.0,22810.0,22525.0,22543.0,24343.0,27117.0,29891.0,31721.0,32022.0
Wyre,18155.0,18028.5,17902.0,19227.0,20064.0,20985.0,23043.0,24953.0,26691.0,27657.0
Wyre Forest,18181.0,18181.0,18158.0,20783.0,20293.0,22665.0,23501.0,24553.0,25605.0,27346.0


In [ ]:
import numpy as np
import pandas as pd

if not isinstance(combined_median.columns, pd.MultiIndex):
    raise ValueError("df.columns must be a MultiIndex")

new_cols = {}

for year in combined_median.columns.levels[0]:

    year_df = combined_median[year]

    values = year_df.values.flatten()
    values = values[~np.isnan(values)]

    low_cut = np.percentile(values, 33)
    high_cut = np.percentile(values, 67)
    base_vals = year_df.mean(axis=1)

    tercile_series = base_vals.apply(
        lambda v: ( "Low" if v < low_cut else "Mid" if v < high_cut else "High" ))
    new_cols[(year, "Tercile")] = tercile_series

tercile_df = pd.DataFrame(new_cols, index=combined_median.index)
tercile_df.columns = pd.MultiIndex.from_tuples(tercile_df.columns, names=combined_median.columns.names)
df_with_tercile = pd.concat([combined_median, tercile_df], axis=1)
df_with_tercile = df_with_tercile.sort_index(axis=1)
df_with_tercile = df_with_tercile.drop(columns=df_with_tercile.columns[df_with_tercile.columns.get_level_values(1) == "50"])
df_with_tercile.to_csv(f"./CleanedData/Income_Tercile_Map.csv", index=True)   



In [ ]:
df_long = combined_percentiles.stack(level=[0,1]).reset_index()
df_long.columns = ['Council', 'Year', 'Percentile', 'Revenue']

df_long['Year'] = pd.to_numeric(df_long['Year'])
df_long['Percentile'] = pd.to_numeric(df_long['Percentile'])
df_long['Revenue'] = pd.to_numeric(df_long['Revenue'])

fig = go.Figure()

x_min, x_max = df_long['Year'].min(), df_long['Year'].max()
y_min, y_max = df_long['Percentile'].min(), df_long['Percentile'].max()
z_min, z_max = df_long['Revenue'].min(), df_long['Revenue'].max()


councils = df_long['Council'].unique()
for i, council in enumerate(councils):
    df_c = df_long[df_long['Council'] == council]
    fig.add_trace(go.Scatter3d(x=df_c['Year'],y=df_c['Percentile'],z=df_c['Revenue'],mode='markers',marker=dict(size=5),name=council, visible=(i==0)))

buttons = []
for i, council in enumerate(councils):
    visible = [False]*len(councils)
    visible[i] = True
    buttons.append(dict(label=council, method="update", args=[{"visible": visible}, {"title": f"Region: {council}"}]))

fig.update_layout(
    updatemenus=[dict(active=0, buttons=buttons, x=1.1, y=0.8)],
    scene=dict(
        xaxis=dict(title='Year', range=[x_min, x_max]),
        yaxis=dict(title='Percentile', range=[y_min, y_max]),
        zaxis=dict(title='Revenue', range=[z_min, z_max]),
    ),
    title="Income Percentiles By Council By Region By Year"
)

fig.show()


/var/folders/dw/62r7df3d26zfcqkyytc46c880000gn/T/ipykernel_90009/2945711680.py:1: FutureWarning: The previous implementation of stack is deprecated and will be removed in a future version of pandas. See the What's New notes for pandas 2.1.0 for details. Specify future_stack=True to adopt the new implementation and silence this warning.
  df_long = combined_percentiles.stack(level=[0,1]).reset_index()
